In [1]:
import sys
sys.path.insert(0,'../..')
from libs.vis_utils import *
from libs.evaluation import *
from libs.ssl_model import VGGSSL
%load_ext autoreload
%autoreload 2

/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
%reload_ext autoreload

In [2]:
data_path = '../data/hbn_preprocessed_windowed_scaled'
checkpoint = 'RP_VGGSSL/checkpoints/epoch=7-step=5064.ckpt'

In [8]:
embs = get_embs(data_path, checkpoint, VGGSSL(), '.')
embs.shape

/home/dung/eeg-ssl/notebooks/RP_VGGSSL_exploration/../../libs/evaluation.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  


(10679, 100)

In [4]:
labels = get_labels('p_factor', data_path)
labels.shape

/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/braindecode/datasets/base.py:514: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_dfs)


(10679,)

In [5]:
subjects = get_labels('subject', data_path)
subjects.shape

/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/braindecode/datasets/base.py:514: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_dfs)


(10679,)

In [6]:
subjects

array(['NDARRD326KB9', 'NDARRD326KB9', 'NDARRD326KB9', ...,
       'NDARNL587LVL', 'NDARNL587LVL', 'NDARNL587LVL'], dtype=object)

In [18]:
rankme(embs)

8.953045

In [41]:
def train_regressor(regr, embs, labels):
    isnan = np.isnan(labels)
    X, Y= embs[~isnan], labels[~isnan]
    assert X.shape[0] == Y.shape[0]
    regr.fit(X, Y)
    score = regr.score(X, Y) 
    return regr, score

In [49]:
def get_prediction_for_subject(subject, embs, labels, regr, subjects):
    subject_embs = embs[subjects==subject]
    subject_labels = labels[subjects==subject]
    return regr.predict(subject_embs).mean(), subject_labels.mean()

def subject_level_score(embs, labels, regr, subjects):
    unique_subjects = np.unique(subjects)
    subject_level_predictions = np.array([get_prediction_for_subject(subject, embs, labels, regr, subjects) for subject in unique_subjects])
    res_sum = ((subject_level_predictions[1] - subject_level_predictions[0])**2).sum()
    total_sum = ((subject_level_predictions[1] - subject_level_predictions[1].mean())**2).sum()
    
    return 1-res_sum/total_sum
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(random_state=1, max_iter=1000)
regressor, sample_level_score = train_regressor(regr, embs, labels)
print(subject_level_score(embs, labels, regressor, subjects), sample_level_score)


-0.12673694155976478 0.515983706430404


In [50]:
from sklearn.linear_model import LinearRegression
regr = LinearRegression()
regressor, sample_level_score = train_regressor(regr, embs, labels)
print(subject_level_score(embs, labels, regressor, subjects), sample_level_score)

0.39130477026591604 0.031188187069314033


In [51]:
from braindecode.datautil import load_concat_dataset
windows_ds = load_concat_dataset(path=data_path, preload=False)